In [283]:
# Import dati dalla prof
from scipy.io import loadmat
import numpy as np
import scipy.linalg as spl
import RisolviSis as RS
import matplotlib.pyplot as plt
dati = loadmat('testE.mat')
A=dati["A"] 
b=dati["b"]


# m = n ?

In [284]:
# Innanzittutto devo verificare se il sistema è sovradeterminato o meno
# m =/= n il sistema è sovradeterminato.
# Inoltre in base ai valori sulle dimensioni posso anche capire quanto sia
# grande o meno la matrice e decidere meglio quale algortimo applicare.
n, m = A.shape
print("Dimensione di A:", n, m)


Dimensione di A: 100 100


# Densa o Sparsa ?
Questo lo faccio solo se m = n.

In [285]:
# Se ho una matrice quadrata controllo se questa sia densa o sparsa:
# se più del 33% degli elementi è diverso da 0 allora la matrice è densa.
n_zeri = np.count_nonzero(A) / (n * m)
perc_n_zeri = n_zeri * 100
print("Percentuale elementi diversi da zero:", perc_n_zeri,"%")


Percentuale elementi diversi da zero: 2.98 %


# Simmetrica ?
Solo se la matrice è grande e sparsa: in questo caso usi i metodi iterativi.

In [286]:
# Se ho una matrice quadrata e grande e sparsa, verifico ora che la matrice sia 
# simmetrica oppure no: in base a questa valutazione capisco quale metodo 
# utilizzare.
# Se la matrice è uguale alla trasposta allora questa sarà definita positiva.
flag = (A == A.T)
if (np.all(flag) == False):
    print("La matrice non è simmetrica")
else:
    print("La matrice è simmetrica e definita positiva")


La matrice non è simmetrica


# Simmetrica == True?
GS SOR, Gradiente, Gradiente coniugato.

# Simmetrica == False?
Controllo se la diagonale è dominante.

# Diagonale dominante?
Jacobi, Gauss-Siedel, GS SOR.

In [287]:
# A questo punto passo ad analizzare la diagonale della matrice: da questo 
# capisco se la diagonale è dominante o meno e se lo è anche in modo stretto.
# A e' a diagonale dominante se il valore assoluto dell'elemento sulla diagonale 
# e' >= della somma in valore in valore assoluto di tutti i valori sulla
# rispettiva riga.
def check_diagonale(A):
    n = A.shape[0]
    flag = True
    for i in range(n):
       diag_elem = np.abs(A[i, i])
       print("elemento diagonale:", diag_elem)
       # per tutte le righe: elementi in riga sommati - elemento diagonale.
       row_sum = np.sum(np.abs(A[i, :])) - np.abs(A[i, i])
       if diag_elem < row_sum:
            print("Matrice non a diagonale dominante")
            flag = False
            return flag
    
    # Se e' a diagonale dominante hai: Jacobi, Gauss-Siedel e GS SOR
    return flag

diag = check_diagonale(A)
print("Matrice a diagonale dominante? ", diag)

elemento diagonale: 2089.7
elemento diagonale: 2088.7
elemento diagonale: 2087.7
elemento diagonale: 2086.7
elemento diagonale: 2085.7
elemento diagonale: 2084.7
elemento diagonale: 2083.7
elemento diagonale: 2082.7
elemento diagonale: 2081.7
elemento diagonale: 2080.7
elemento diagonale: 2079.7
elemento diagonale: 2078.7
elemento diagonale: 2077.7
elemento diagonale: 2076.7
elemento diagonale: 2075.7
elemento diagonale: 2074.7
elemento diagonale: 2073.7
elemento diagonale: 2072.7
elemento diagonale: 2071.7
elemento diagonale: 2070.7
elemento diagonale: 2069.7000000000003
elemento diagonale: 2068.7000000000003
elemento diagonale: 2067.7
elemento diagonale: 2066.7
elemento diagonale: 2065.7
elemento diagonale: 2064.7
elemento diagonale: 2063.7
elemento diagonale: 2062.7
elemento diagonale: 2061.7000000000003
elemento diagonale: 2060.7
elemento diagonale: 2059.7
elemento diagonale: 2058.7
elemento diagonale: 2057.7
elemento diagonale: 2056.7
elemento diagonale: 2055.7
elemento diagonale:

# Piccola e Densa ?
Ti affidi ai metodi di fattorizzazione:
- QR: sempre possibile.
- LU: determinante di A diverso da 0.
- Cholesky: esiste per matrici simmetriche e definite positive.

# Simmetria ?

In [288]:
# Se la matrice è uguale alla trasposta allora questa sarà definita positiva.
flag = (A == A.T)
if (np.all(flag) == False):
    print("La matrice non è simmetrica")
else:
    print("La matrice è simmetrica e definita positiva")


La matrice non è simmetrica


# Sistemi sovradeterminati

In [289]:
import scipy.linalg as spLin
dati = loadmat('testF.mat')
A1=dati["A"] 
b1=dati["b"]

# m > n ?

In [290]:
m, n = A1.shape
print("Dimensione di A", m ,n)

Dimensione di A 50 19


# Rango massimo ?

In [291]:
# Vedo se il determinante di A è diverso da 0
rank = np.linalg.matrix_rank(A)
if(A.shape[0] == rank):
    print("Rango massimo e il determinante è diverso da 0.")
else:
    print("Rango non massimo e il determinante è nullo.")


Rango massimo e il determinante è diverso da 0.


# Rango =/= max: 
SVDLS

# Rango = max
Calcolo l'indice di condizionamento della matrice.

In [292]:
ind_cond = np.linalg.cond(A1)
print(ind_cond)

if(ind_cond < A1.shape[0]**3):
    print("Matrice ben condizionata")
elif(ind_cond < A1.shape[0]**10):
    # anche se la vera condizione è un'altra
    print("Matrice mediamente mal condizionata")
else:
    print("Matrice mal condizionata")


5.1781251848074726e+17
Matrice mal condizionata


# Matrice ben condizionata ?
Equazioni normali

# Rango max e Mediamente mal condizionata ?
QRLS